In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!! pip install open-metric-learning
!! pip install pytorch-metric-learning
!! pip install pillow==9.0.0

  Attempting uninstall: triton
    Found existing installation: triton 2.3.0
    Uninstalling triton-2.3.0:
      Successfully uninstalled triton-2.3.0
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0+cu121
    Uninstalling torch-2.3.0+cu121:
      Successfully uninstalled torch-2.3.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.18.0+cu121
    Uninstalling torchvision-0.18.0+cu121:
      Successfully uninstalled torchvision-0.18.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.1.1 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 2.1.1 which is incompatible.


['Collecting pillow==9.0.0',
 '  Downloading Pillow-9.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/4.3 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/4.3 MB\x1b \x1b628.8 kB/s\x1b eta \x1b0:00:07\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.1/4.3 MB\x1b \x1b893.6 kB/s\x1b eta \x1b0:00:05\x1b',
 '\x1b[2K     \x1b━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.1/4.3 MB\x1b \x1b1.5 MB/s\x1b eta \x1b0:00:03\x1b',
 '\x1b[2K     \x1b━━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.3/4.3 MB\x1b \x1b2.3 MB/s\x1b eta \x1b0:00:02\x1b',
 '\x1b[2K     \x1b━━━━━━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.7/4.3 MB\x1b \x1b4.0 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b1.4/4.3 MB\x1b \

In [ ]:
import torch
from tqdm import tqdm

from oml.datasets.base import DatasetWithLabels
from oml.losses.triplet import TripletLossWithMiner
from oml.miners.inbatch_all_tri import AllTripletsMiner
from oml.models import ViTExtractor
from oml.samplers.balance import BalanceSampler
from oml.datasets.base import DatasetQueryGallery
from oml.metrics.embeddings import EmbeddingMetrics
from oml.utils.dataframe_format import check_retrieval_dataframe_format
from oml.utils.misc_torch import PCA
from oml.inference.flat import inference_on_images
from oml.registry.transforms import get_transforms_for_pretrained
from oml.utils.misc_torch import pairwise_dist
from pytorch_metric_learning import losses, distances, reducers, miners
import pandas as pd
from oml.transforms.images.torchvision import get_normalisation_resize_torch
from oml.models.meta.siamese import ConcatSiamese
from oml.miners.pairs import PairsMiner
from oml.inference.flat import inference_on_dataframe
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader
from oml.retrieval.postprocessors.pairwise import PairwiseImagesPostprocessor
from pprint import pprint



In [ ]:
import pandas as pd
from pathlib import Path
import pathlib

iris_path='/content/data/iris_rf'
iris_path=pathlib.PosixPath(iris_path)
p=iris_path

iris_id_csv="/content/drive/MyDrive/Стажировка/Зеница/df_iris_rf_LR.csv"
# open metric learning format for datasets and pipelines...
df_iris_id = pd.read_csv(iris_id_csv)

In [ ]:
import gdown
url = 'https://drive.google.com/drive/folders/1-0Du_tEDXGH6TmXW_aoMpHkX48PnnBgH?usp=sharing'
gdown.download_folder(url, quiet=True)

['/content/models/iris-D_rf.zip']

In [ ]:
!mkdir data
!unzip -q '/content/models/iris-D_rf.zip' -d '/content/data'

mkdir: cannot create directory ‘data’: File exists


<H3>Формируем датафреймы с 3мя базами

In [ ]:
df_iris_id = df_iris_id.drop(['eye', 'filename'], axis=1)

In [ ]:
df_iris_id

,label,path,split,query,gallery
0,13,014L_2_png.rf.c8eee382085dc0bc5c97fb15bea99f21...,train,NaN,NaN
1,10,011R_1_png.rf.cadb3001384a511af6c28357f23d7225...,validation,1.0,0.0
2,21,022L_3_png.rf.61348dbe7f385399a470a3c88cb41956...,train,NaN,NaN
3,27,028R_3_png.rf.78619f0cf6f3dbbf3e3edf076a9a7b16...,train,NaN,NaN
4,19,020L_1_png.rf.8b71bc5cc7b69610fa0e1281756b012e...,train,NaN,NaN
...,...,...,...,...,...
917,46,047L_3_png.rf.0773ccadad514324b10302744216b65c...,train,NaN,NaN
918,59,060R_2_png.rf.c48f50d1d1ea2a4f4d5dc9f16ef24fc6...,train,NaN,NaN
919,57,058R_2_png.rf.c4f6c7b5d8148bcfb20de45067684c29...,train,NaN,NaN
920,6,007R_1_png.rf.b2e2a37d43f307178032e76dbcac8816...,train,NaN,NaN


<H3> Iris_id dataset

<h3> Теперь берем предобученную на огромном датасете (что то из торговли) модель для эмбеддингов и тренируем ее 1 эпоху (больше не надо!) на iris_id train

In [ ]:
model_types=['vits16_dino', 'vits8_dino', 'vitb16_dino', 'vitb8_dino', 'vits16_inshop', 'vits16_sop', 'vits16_cub', 'vits16_cars']
modeltype='vits16_dino'
arch='vits16'

In [ ]:
import torch
device = torch.device("cuda")

<H3> Валидность датасета проверяется тут

In [ ]:
from pathlib import Path
from oml.utils.dataframe_format import check_retrieval_dataframe_format
p=Path(iris_path)

check_retrieval_dataframe_format(df=df_iris_id, dataset_root=p)

AssertionError: Index(['label', 'path', 'split', 'query', 'gallery'], dtype='object')

<H1> Из своих обученных берем веса

In [ ]:
from oml.const import CKPT_SAVE_ROOT as CKPT_DIR, MOCK_DATASET_PATH as DATA_DIR
from oml.models import ViTExtractor
from oml.registry.transforms import get_transforms_for_pretrained


# first time
model = ViTExtractor.from_pretrained("vits16_dino")
# second time
#model = ViTExtractor(weights=CKPT_DIR / "vits16_dino.ckpt", arch="vits16", normalise_features=False)
transform, im_reader = get_transforms_for_pretrained("vits16_dino")

https://dl.fbaipublicfiles.com/dino/dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth
Checkpoint is already here.


<H3>Обучение с нуля (но предобученные веса загружаются автоматом)

Triple loss on embeddings

<H3>одна эпоха

In [ ]:
# тут НЕ сохраненные на зрачках веса грузятся
#model = ViTExtractor(modeltype, arch=arch, normalise_features=False).train()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
#transform = get_normalisation_resize_torch(im_size=im_size)
train_dataset = DatasetWithLabels(df, dataset_root=p,transform=transform)
device = torch.device("cuda")
model.to(device)

# PML specific
distance = distances.LpDistance(p=2)
reducer = reducers.ThresholdReducer(low=0)
criterion = losses.TripletMarginLoss()

miner = miners.TripletMarginMiner(margin=threshold, distance=distance, type_of_triplets="hard")

sampler = BalanceSampler(train_dataset.get_labels(), n_labels=2, n_instances=2)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=sampler)

# Одна эпоха потом посмотрим когда метрики построим может и еще надо
n_epoch=1

for i in range(n_epoch):
  for batch in tqdm(train_loader):
    embeddings = model(batch["input_tensors"].to(device))
    loss = criterion(embeddings, batch["labels"], miner(embeddings, batch["labels"]))  # PML specific
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()




100%|██████████| 32/32 [00:42<00:00,  1.33s/it]


<H3> По слегка оттренированной
 модели смотрим какие метрики в валидации получились
 в калькуляторе

In [ ]:
miner = miners.TripletMarginMiner(margin=threshold, distance=distance, type_of_triplets="all")

In [ ]:
# создаем калькулятор метрик, в котором будем накапливать вектора

val_dataset = DatasetQueryGallery(df, dataset_root=p,transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=4)

calculator = EmbeddingMetrics(map_top_k=(5,3,2,1),cmc_top_k =(5,3,2,1),precision_top_k =(5,3,2,1) )
calculator.setup(num_samples=len(train_dataset))

with torch.no_grad():
    for batch in tqdm(val_loader):
        batch["embeddings"] = model(batch["input_tensors"].to(device))
        calculator.update_data(batch)  # накапливаем вектора



100%|██████████| 96/96 [01:11<00:00,  1.35it/s]


In [ ]:
metrics = calculator.compute_metrics()  # вычисляем метрики: cmc@k, precision@k, map@k


Metrics:
{'OVERALL': {'cmc': {1: tensor(0.9961),
                     2: tensor(1.),
                     3: tensor(1.),
                     5: tensor(1.)},
             'map': {1: tensor(0.9961),
                     2: tensor(0.9980),
                     3: tensor(0.9938),
                     5: tensor(0.9808)},
             'pcf': {0.5: tensor(0.0078)},
             'precision': {1: tensor(0.9961),
                           2: tensor(0.9766),
                           3: tensor(0.9362),
                           5: tensor(0.9434)}}}


<H3> Метрики свиду ОТЛИЧНЫЕ - спасибо за веса автору

Теперь вместо поиска ближайшего по расстоянию
поищем ближайшего по методу KNN (K neaarest neibourgs) при
top_k = 3
получим всего по одной ошибке на тестовых датасетах!

<H3>Инференс по  базе MMU

In [ ]:
df=df_iris_amf

In [ ]:
# Смысл такой что по запросам query мы ищем ближайший (похоже 1)
# но только среди элементов gallery
queries=[]
df_val=df[df['split']=='validation']
df_val["path"] = df_val["path"].apply(lambda x: p / x)
queries = df_val[df_val["is_query"]==1.0]["path"].tolist()
galleries = df_val[df_val["is_gallery"]==1.0]["path"].tolist()

args = {"num_workers": 0, "batch_size": 8}
# эмбеддинги запросов и галереи
features_queries = inference_on_images(model, paths=queries, transform=transform, **args)
features_galleries = inference_on_images(model, paths=galleries, transform=transform, **args)

use_knn = True
top_k = 3
if use_knn:
    from sklearn.neighbors import NearestNeighbors
    knn = NearestNeighbors(algorithm="auto", p=2)
    knn.fit(features_galleries)
    dists, ii_closest = knn.kneighbors(features_queries, n_neighbors=top_k, return_distance=True)

else:
    dist_mat = pairwise_dist(x1=features_queries, x2=features_galleries)
    dists, ii_closest = torch.topk(dist_mat, dim=1, k=top_k, largest=False)

print(f"Top {top_k} items closest to queries are:\n {ii_closest}")


In [ ]:
dists

In [ ]:
df_queries = df_val[df_val["is_query"]==1.0]
df_galleries = df_val[df_val["is_gallery"]==1.0]

In [ ]:
ii_closest[0].tolist()

[0, 1, 104]

In [ ]:
ppath1=[]
ppath2=[]
ppath3=[]
predict1=[]
predict2=[]
predict3=[]
dist1=[]
dist2=[]
dist3=[]



for i3,m3 in zip(ii_closest.tolist(),dists.tolist()):
  print(i3)
  print(m3)
  predict1.append(df_galleries.iloc[i3[0]]['label'])
  predict2.append(df_galleries.iloc[i3[1]]['label'])
  predict3.append(df_galleries.iloc[i3[2]]['label'])
  ppath1.append((df_galleries.iloc[i3[0]]['src_file']))
  ppath2.append((df_galleries.iloc[i3[1]]['src_file']))
  ppath3.append((df_galleries.iloc[i3[2]]['src_file']))
  dist1.append(m3[0])
  dist2.append(m3[1])
  dist3.append(m3[2])





In [ ]:
df_queries['predict']=predict1
df_queries['predict2']=predict2
df_queries['predict3']=predict3
df_queries['ppath']=ppath1
df_queries['ppath2']=ppath2
df_queries['ppath3']=ppath3
df_queries['dist']=dist1
df_queries['dist2']=dist2
df_queries['dist3']=dist3

In [ ]:
df_match=df_queries[df_queries['label']==df_queries['predict']]

In [ ]:
df_match

,label,src_file,split,is_query,is_gallery,eye,ds,path,predict,predict2,predict3,ppath,ppath2,ppath3,dist,dist2,dist3
0,0,Iris_20220818_122827_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,0,0,0,Iris_20220818_122823_Left.bmp,Iris_20220818_122821_Left.bmp,Iris_20220818_122825_Left.bmp,10.258099,10.711012,11.534957
3,0,Iris_20220818_122818_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,0,0,0,Iris_20220818_122823_Left.bmp,Iris_20220818_122821_Left.bmp,Iris_20220818_122825_Left.bmp,10.615119,11.424499,12.894082
6,1,Iris_20220818_105655_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,1,1,1,Iris_20220818_105659_Left.bmp,Iris_20220818_105650_Left.bmp,Iris_20220818_105652_Left.bmp,15.679811,18.809160,20.351361
9,1,Iris_20220818_105657_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,1,1,1,Iris_20220818_105659_Left.bmp,Iris_20220818_105650_Left.bmp,Iris_20220818_105652_Left.bmp,14.519103,16.641416,17.637912
10,2,Iris_20220818_121801_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,2,2,2,Iris_20220818_121757_Left.bmp,Iris_20220818_121804_Left.bmp,Iris_20220818_121804_Right.bmp,16.792654,28.866051,35.092733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,51,Iris_20220817_123808_Right.bmp,validation,1.0,0.0,R,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/z...,51,51,51,Iris_20220817_123810_Right.bmp,Iris_20220817_123757_Right.bmp,Iris_20220817_123805_Right.bmp,12.635336,20.095952,21.438480
531,52,Iris_20220818_122133_Right.bmp,validation,1.0,0.0,R,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/z...,52,52,52,Iris_20220818_122136_Right.bmp,Iris_20220818_122132_Right.bmp,Iris_20220818_122127_Right.bmp,18.537797,25.477375,35.512447
533,52,Iris_20220818_122130_Right.bmp,validation,1.0,0.0,R,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/z...,52,52,52,Iris_20220818_122127_Right.bmp,Iris_20220818_122132_Right.bmp,Iris_20220818_122136_Right.bmp,17.688476,26.436992,27.788212
536,53,Iris_20220818_122420_Right.bmp,validation,1.0,0.0,R,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/z...,53,53,53,Iris_20220818_122416_Right.bmp,Iris_20220818_122413_Right.bmp,Iris_20220818_122405_Right.bmp,9.990708,12.787806,15.351968


In [ ]:
df_err=df_queries[df_queries['label']!=df_queries['predict']]

In [ ]:
df_err

,label,src_file,split,is_query,is_gallery,eye,ds,path,predict,predict2,predict3,ppath,ppath2,ppath3,dist,dist2,dist3
23,4,Iris_20220817_115959_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,0,31,31,Iris_20220818_122825_Left.bmp,Iris_20220818_122912_Left.bmp,Iris_20220818_122919_Left.bmp,36.641959,37.163879,37.250195


<H3> Все ниже можно не читать - это исследование на предмет не слишком ли у нас большой эмбеддинг - 384)

In [ ]:
df=df_iris_amf

In [ ]:
# Смысл такой что по запросам query мы ищем ближайший (похоже 1)
# но только среди элементов gallery
queries=[]
df_val=df[df['split']=='validation']
df_val["path"] = df_val["path"].apply(lambda x: p / x)
queries = df_val[df_val["is_query"]==1.0]["path"].tolist()
galleries = df_val[df_val["is_gallery"]==1.0]["path"].tolist()

args = {"num_workers": 0, "batch_size": 8}
# эмбеддинги запросов и галереи
features_queries = inference_on_images(model, paths=queries, transform=transform, **args)
features_galleries = inference_on_images(model, paths=galleries, transform=transform, **args)

use_knn = True
top_k = 3
if use_knn:
    from sklearn.neighbors import NearestNeighbors
    knn = NearestNeighbors(algorithm="auto", p=2)
    knn.fit(features_galleries)
    dists, ii_closest = knn.kneighbors(features_queries, n_neighbors=top_k, return_distance=True)

else:
    dist_mat = pairwise_dist(x1=features_queries, x2=features_galleries)
    dists, ii_closest = torch.topk(dist_mat, dim=1, k=top_k, largest=False)

print(f"Top {top_k} items closest to queries are:\n {ii_closest}")


In [ ]:
df_queries = df_val[df_val["is_query"]==1.0]
df_galleries = df_val[df_val["is_gallery"]==1.0]

In [ ]:
ppath1=[]
ppath2=[]
ppath3=[]
predict1=[]
predict2=[]
predict3=[]
dist1=[]
dist2=[]
dist3=[]



for i3,m3 in zip(ii_closest.tolist(),dists.tolist()):
  print(i3)
  print(m3)
  predict1.append(df_galleries.iloc[i3[0]]['label'])
  predict2.append(df_galleries.iloc[i3[1]]['label'])
  predict3.append(df_galleries.iloc[i3[2]]['label'])
  ppath1.append((df_galleries.iloc[i3[0]]['src_file']))
  ppath2.append((df_galleries.iloc[i3[1]]['src_file']))
  ppath3.append((df_galleries.iloc[i3[2]]['src_file']))
  dist1.append(m3[0])
  dist2.append(m3[1])
  dist3.append(m3[2])


In [ ]:
df_queries['predict']=predict1
df_queries['predict2']=predict2
df_queries['predict3']=predict3
df_queries['ppath']=ppath1
df_queries['ppath2']=ppath2
df_queries['ppath3']=ppath3
df_queries['dist']=dist1
df_queries['dist2']=dist2
df_queries['dist3']=dist3

<ipython-input-71-00e9fe507c25>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_queries['predict']=predict1
<ipython-input-71-00e9fe507c25>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_queries['predict2']=predict2
<ipython-input-71-00e9fe507c25>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [ ]:
df_match=df_queries[df_queries['label']==df_queries['predict']]

In [ ]:
df_match

,label,src_file,split,is_query,is_gallery,eye,ds,path,predict,predict2,predict3,ppath,ppath2,ppath3,dist,dist2,dist3
0,0,Iris_20220818_122827_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,0,0,0,Iris_20220818_122823_Left.bmp,Iris_20220818_122821_Left.bmp,Iris_20220818_122825_Left.bmp,10.258099,10.711012,11.534957
3,0,Iris_20220818_122818_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,0,0,0,Iris_20220818_122823_Left.bmp,Iris_20220818_122821_Left.bmp,Iris_20220818_122825_Left.bmp,10.615119,11.424499,12.894082
6,1,Iris_20220818_105655_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,1,1,1,Iris_20220818_105659_Left.bmp,Iris_20220818_105650_Left.bmp,Iris_20220818_105652_Left.bmp,15.679811,18.809160,20.351361
9,1,Iris_20220818_105657_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,1,1,1,Iris_20220818_105659_Left.bmp,Iris_20220818_105650_Left.bmp,Iris_20220818_105652_Left.bmp,14.519103,16.641416,17.637912
10,2,Iris_20220818_121801_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,2,2,2,Iris_20220818_121757_Left.bmp,Iris_20220818_121804_Left.bmp,Iris_20220818_121804_Right.bmp,16.792654,28.866051,35.092733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,51,Iris_20220817_123808_Right.bmp,validation,1.0,0.0,R,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/z...,51,51,51,Iris_20220817_123810_Right.bmp,Iris_20220817_123757_Right.bmp,Iris_20220817_123805_Right.bmp,12.635336,20.095952,21.438480
531,52,Iris_20220818_122133_Right.bmp,validation,1.0,0.0,R,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/z...,52,52,52,Iris_20220818_122136_Right.bmp,Iris_20220818_122132_Right.bmp,Iris_20220818_122127_Right.bmp,18.537797,25.477375,35.512447
533,52,Iris_20220818_122130_Right.bmp,validation,1.0,0.0,R,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/z...,52,52,52,Iris_20220818_122127_Right.bmp,Iris_20220818_122132_Right.bmp,Iris_20220818_122136_Right.bmp,17.688476,26.436992,27.788212
536,53,Iris_20220818_122420_Right.bmp,validation,1.0,0.0,R,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/z...,53,53,53,Iris_20220818_122416_Right.bmp,Iris_20220818_122413_Right.bmp,Iris_20220818_122405_Right.bmp,9.990708,12.787806,15.351968


In [ ]:
df_err=df_queries[df_queries['label']!=df_queries['predict']]

In [ ]:
df_err

,label,src_file,split,is_query,is_gallery,eye,ds,path,predict,predict2,predict3,ppath,ppath2,ppath3,dist,dist2,dist3
23,4,Iris_20220817_115959_Left.bmp,validation,1.0,0.0,L,amf,/content/drive/MyDrive/iris/AMF_Iris_Dataset/a...,0,31,31,Iris_20220818_122825_Left.bmp,Iris_20220818_122912_Left.bmp,Iris_20220818_122919_Left.bmp,36.641959,37.163879,37.250195


<H3>Другие модели пока не проверял кром vits_16_dino  (смотрите modeltype)

In [ ]:
print(list(ViTExtractor.pretrained_models.keys()))

['vits16_dino', 'vits8_dino', 'vitb16_dino', 'vitb8_dino', 'vits16_inshop', 'vits16_sop', 'vits16_cub', 'vits16_cars']


In [ ]:
modelpathLR='/content/drive/MyDrive/iris/iris_dataset_LRknn1.pt'
torch.save(model.state_dict(), modelpathLR)

In [ ]:
cpu = torch.device("cpu")

In [ ]:
mcpu=model.to(cpu)

In [ ]:
modelpathLRcpu='/content/drive/MyDrive/iris/iris_dataset_LRknn1_cpu.pt'
torch.save(mcpu.state_dict(), modelpathLRcpu)